In [32]:
# links

# acs
# https://api.census.gov/data/2018/acs/acs5/profile/examples.html
# https://api.census.gov/data/2018/acs/acs5/profile/geography.html
# https://api.census.gov/data/2018/acs/acs5/variables.html
# https://data.census.gov/cedsci/table?q=United%20States
# https://www.census.gov/data/developers/data-sets/acs-5year.html

# qwi
# https://www.census.gov/data/developers/data-sets/qwi.html
# https://api.census.gov/data/timeseries/qwi/sa/variables.html
# https://api.census.gov/data/timeseries/qwi/sa/examples.html
# https://api.census.gov/data/timeseries/qwi.html
# https://lehd.ces.census.gov/doc/QWIPU_Data_Schema.pdf
# https://lehd.ces.census.gov/doc/QWI_101.pdf

In [33]:
%pwd

'C:\\Users\\Tola\\Documents\\Github\\capstone'

In [34]:
%cd Documents/Github/capstone

[WinError 3] The system cannot find the path specified: 'Documents/Github/capstone'
C:\Users\Tola\Documents\Github\capstone


In [35]:
#manipulate dataframes/geodataframes in python
import pandas as pd
import geopandas as gpd

#make API calls with python
import requests

#allows us to store results of API call cleanly
import json

In [36]:
import numpy as np
from shapely.geometry import Point, Polygon
from fiona.crs import from_epsg
import matplotlib.pyplot as plt
%matplotlib inline

In [37]:
from ipyleaflet import Map, GeoData, GeoJSON, basemaps, basemap_to_tiles, Icon, Circle, Marker, LayerGroup, WidgetControl
from ipywidgets import Button 
import ipywidgets as widgets

In [38]:
#drawing basic map
center = (40.7210907,-73.9877836)
basemap = basemap_to_tiles(basemaps.CartoDB.Positron)

icon = Icon(icon_url='icon.png', icon_size=[15, 15])
marker = Marker(location=center, draggable=False, icon=icon, opacity = 0.8)

m = Map(layers=(basemap, ), center=center, zoom=15, min_zoom = 11, max_zoom = 20)

###HALF-MI BUFFER
#set location of center as location of the point
markerlocation=marker.location
print(markerlocation)
    
radius = 402 #in-meters
color = "black"
fill_color = "white"
fill_opacity = .2
weight = 2

##click on map to add marker, removes old marker as well
def handle_click(**kwargs):
    if kwargs.get('type') == 'click':
        layer_group.clear_layers();
        marker = Marker(location=kwargs.get('coordinates'), draggable=False, icon=icon, opacity = 0.8)
        layer_group.add_layer(marker)
        
        #updating the circle's location
        markerlocation=marker.location
        circle.location = markerlocation
        layer_group.add_layer(circle);
        
m.on_interaction(handle_click)
######

#create half-mi buffer
circle = Circle()
circle.location = markerlocation
circle.radius = radius
circle.color = color
circle.fill_color = fill_color
circle.fill_opacity = fill_opacity
circle.weight = weight
###########

    
# def on_location_changed(event):
# #remove any circles existing first
#     layer_group.clear_layers();

# #update location of center as location of the point
#     markerlocation = marker.location
#     circle.location = markerlocation
#     layer_group.add_layer(circle);
        
# pass
# marker.observe(on_location_changed, 'location')
# #######


#adding button that performs the analysis on click 
button = Button(description="Analyze")

def on_button_clicked(b):   
    #extracting the marker location for the buffer
    location=circle.location
    
    df2 = pd.DataFrame(location)
    print(df2)

    df=df2.transpose()
    df.columns=['Latitude','Longitude']
    print(df.head())

    #creating a geodataframe from the point
    gdf = gpd.GeoDataFrame(
        df, geometry=gpd.points_from_xy(df.Longitude, df.Latitude))
#creating circle for cut
    # gdf.crs = 'EPSG:4326'
    gdf.crs = from_epsg(3857)
    print(gdf.crs)
    test=gdf.copy()
    #making buffer
    test['geometry'] = test.geometry.buffer(.004,  cap_style=1, join_style=1)
    # change back
    test.crs = from_epsg(3857)
#creating rectangle for cut
    rect_orig=gdf.copy()
    rect_orig['geometry'] = rect_orig.geometry.buffer(4,  cap_style=1, join_style=1)
#difference
    diff = gpd.overlay(rect_orig, test, how='difference')

    geo_data = GeoData(geo_dataframe = diff,
                   style={'fillColor': 'black', 'color':'black', 'weight':2,'fillOpacity':0.5},
#                    hover_style={'fillColor': 'red' , 'fillOpacity': 0.2},
                   name = 'Boundary')
    layer_group.add_layer(geo_data)
    
button.on_click(on_button_clicked)

    
widget_control = WidgetControl(widget=button, position='bottomleft')
m.add_control(widget_control)


layer_group = LayerGroup(layers=(marker, circle))
m.add_layer(layer_group)

display(m)

[40.7210907, -73.9877836]


Map(center=[40.7210907, -73.9877836], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_titl…

           0
0  40.716194
1 -74.000992
    Latitude  Longitude
0  40.716194 -74.000992
{'init': 'epsg:3857', 'no_defs': True}
           0
0  40.720943
1 -73.993780
    Latitude  Longitude
0  40.720943  -73.99378
{'init': 'epsg:3857', 'no_defs': True}


In [ ]:
##what state does this fall in?


In [ ]:
##Adding in MSAs and Census Tracts for states 

In [26]:
zipped_shp_url = 'https://www2.census.gov/geo/tiger/TIGER2019/STATE/tl_2019_us_state.zip'

all_ca_tract_geo = zip_shp_to_gdf(zipped_shp_url)
alameda_tract_geo = all_ca_tract_geo[all_ca_tract_geo.COUNTYFP=='001']
alameda_tract_geo = alameda_tract_geo.set_index("GEOID")['geometry'].to_crs(epsg=3310)
alameda_tract_geo.head()

NameError: name 'zip_shp_to_gdf' is not defined

In [29]:
import geojson

In [30]:
%%time
endpoint = 'https://tigerweb.geo.census.gov/arcgis/rest/services/TIGERweb/tigerWMS_ACS2015/MapServer/84/query'
s = requests.session()
s.params = {
    'where': '',
    'text': '',
    'objectIds': '',
    'time': '',
    'geometry': '[[-2336502.536,-20037700.],[1.1615796196E7,2.0656504635E7]]',
    'geometryType': 'esriGeometryEnvelope',
    'inSR': 102100, # EPSG 3857
    'spatialRel': 'esriSpatialRelIntersects',
    'relationParam': '',
    'outFields': 'STATE,COUNTY,BASENAME,NAME,STGEOMETRY',
    'returnGeometry': True,
    'returnTrueCurves': False,
    'maxAllowableOffset': '',
    'geometryPrecision': '',
    'outSR': '',
    'returnIdsOnly': False,
    'returnCountOnly': False,
    'orderByFields': '',
    'groupByFieldsForStatistics': '',
    'outStatistics': '',
    'returnZ': False,
    'returnM': False,
    'gdbVersion': '',
    'returnDistinctValues': False,
    'f': 'geojson',        
}
start = 0
done = False
features = []
crs = None
while not done:
    r = s.get(endpoint, params={
        'resultOffset': start,
        'resultRecordCount': 32,
    })
    r.raise_for_status()
    counties = geojson.loads(r.text)
    newfeats = counties.__geo_interface__['features']
    if newfeats:
        features.extend(newfeats)
        crs=counties.__geo_interface__['crs']
        start += len(newfeats)
        print("Received", len(newfeats), "entries,", start, "total")
    else:
        done = True
counties_df = gpd.GeoDataFrame.from_features(features, crs=crs)

Received 32 entries, 32 total
Received 32 entries, 64 total
Received 32 entries, 96 total
Received 32 entries, 128 total
Received 32 entries, 160 total
Received 32 entries, 192 total
Received 32 entries, 224 total
Received 32 entries, 256 total
Received 32 entries, 288 total
Received 32 entries, 320 total
Received 32 entries, 352 total
Received 32 entries, 384 total
Received 32 entries, 416 total
Received 32 entries, 448 total
Received 32 entries, 480 total
Received 32 entries, 512 total
Received 32 entries, 544 total
Received 32 entries, 576 total
Received 32 entries, 608 total
Received 32 entries, 640 total
Received 32 entries, 672 total
Received 32 entries, 704 total
Received 32 entries, 736 total
Received 32 entries, 768 total
Received 32 entries, 800 total
Received 32 entries, 832 total
Received 32 entries, 864 total
Received 32 entries, 896 total
Received 32 entries, 928 total
Received 32 entries, 960 total
Received 32 entries, 992 total
Received 32 entries, 1024 total
Received 3

In [28]:
counties_df

NameError: name 'counties_df' is not defined

In [ ]:
# key = 9330dc4bf086a84f19fb412bb15f232507301de6
with open('census_key.txt') as key:
    api_key=key.read().strip()
 
# here is defining the variables...
NAME_acs='B01003_001E,B25074_001E'
state='36'
county='005,047,061'
# tract='001002'
time='2019-Q2'
NAME_qwi='geography,industry,sex,agegrp,education,firmsize'
MSA_short='metropolitan%20statistical%20area/micropolitan%20statistical%20area'

In [ ]:
acs_url = f'https://api.census.gov/data/2018/acs/acs5/'
qwi_url = f'https://api.census.gov/data/timeseries/qwi/sa'

In [ ]:
# 	https://api.census.gov/data/timeseries/qwi/sa?get=year,agegrp,quarter&for=metropolitan%20statistical%20area/micropolitan%20statistical%20area:*&in=state:24&time=2010-Q1&key=YOUR_KEY_GOES_HERE

# api call using variables instead of directly is way easier
qwi_data_url = f'{qwi_url}?&get={NAME_qwi}&for={MSA_short}:*&in=state:{state}&time={time}&key={api_key}'
response_qwi=requests.get(qwi_data_url)
print(response_qwi.text)

In [ ]:
# https://api.census.gov/data/2018/acs/acs5/profile?get=DP04_0001E&for=tract:*&in=state:36%20county:005,047&key=9330dc4bf086a84f19fb412bb15f232507301de6

# api call using variables instead of directly is way easier
acs_data_url = f'{acs_url}?&get={NAME_acs}&for=tract:*&in=state:{state}%20county:{county}&key={api_key}'
response_acs=requests.get(acs_data_url)
print(response_acs.text)

In [ ]:
#load the response into a JSON
data_acs=response_acs.json()
#ignoring the first element which is column names, replacing columns with the columns from the table
acs=pd.DataFrame(data_acs[1:], columns=data_acs[0])

acs.head()

In [ ]:
#load the response into a JSON
data_qwi=response_qwi.json()
#ignoring the first element which is just column names, replacing columns with the columns from the table
qwi=pd.DataFrame(data_qwi[1:], columns=data_qwi[0])

qwi.head()

In [ ]:
#proportional split
